In [ ]:
!pip install -U ultralytics
!pip install -U ipywidgets

Splitting the dataset:
train ratio: 50%
val ratio: 25%
test ratio: 25%

In [ ]:
import os
import shutil

dataset_path = '/kaggle/input/chromosome-yolo-dataset/yolo_dataset'

output_path = '/kaggle/working'

splits = ['train', 'val', 'test']

for split in splits:
    src_img_dir = os.path.join(dataset_path, 'images', split)
    src_lbl_dir = os.path.join(dataset_path, 'labels', split)

    dst_img_dir = os.path.join(output_path, split, 'images')
    dst_lbl_dir = os.path.join(output_path, split, 'labels')

    os.makedirs(dst_img_dir, exist_ok=True)
    os.makedirs(dst_lbl_dir, exist_ok=True)

    for fname in os.listdir(src_img_dir):
        if not fname.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        shutil.copy(os.path.join(src_img_dir, fname),
                    os.path.join(dst_img_dir, fname))

        label_name = os.path.splitext(fname)[0] + '.txt'
        label_src = os.path.join(src_lbl_dir, label_name)

        if os.path.exists(label_src):
            shutil.copy(label_src, os.path.join(dst_lbl_dir, label_name))
        else:
            print(f'Sem label para {fname}')

print("Cópia concluída para /kaggle/working/{train,val,test}")

In [ ]:
import os

working_path = '/kaggle/working'

def count_images(split):
    images_dir = os.path.join(working_path, split, 'images')
    return len([f for f in os.listdir(images_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])

train_count = count_images('train')
val_count = count_images('val')
test_count = count_images('test')
total_count = train_count + val_count + test_count

print(f"Total de imagens em /kaggle/working: {total_count}")
print(f"Imagens de treino: {train_count}")
print(f"Imagens de validação: {val_count}")
print(f"Imagens de teste: {test_count}")


In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

dataset_path = '/kaggle/input/chromosome-yolo-dataset/yolo_dataset'
split = 'test'
image_id = '7.png'

if not image_id.endswith('.png'):
    image_id += '.png'

image_path = os.path.join(dataset_path, 'images', split, image_id)
label_path = os.path.join(dataset_path, 'labels', split, image_id.replace('.png', '.txt'))

image = cv2.imread(image_path)
if image is None:
    raise FileNotFoundError(f'Imagem não encontrada: {image_path}')

image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
h, w, _ = image.shape

if os.path.exists(label_path):
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue

            cls_id, x_center, y_center, bw, bh = map(float, parts)

            # YOLO → pixel
            x_center *= w
            y_center *= h
            bw *= w
            bh *= h

            x1 = int(x_center - bw / 2)
            y1 = int(y_center - bh / 2)
            x2 = int(x_center + bw / 2)
            y2 = int(y_center + bh / 2)

            cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(image, f"Class {int(cls_id)}", (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
else:
    print(f'Nenhum label encontrado: {label_path}')

plt.figure(figsize=(8, 8))
plt.imshow(image)
plt.axis('off')
plt.title(f"{split.upper()} - {image_id}")
plt.show()

In [ ]:
import torch
from ultralytics import YOLO
from torch import nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = YOLO('yolo11m.pt')

device_ids = [0, 1] 

config_path = '/kaggle/input/config/config.yaml'

results = model.train(
    data=config_path,
    epochs=200,
    batch=16,
    imgsz=1080,
    device=device_ids
)

In [ ]:
metrics = model.val(data=config_path, split="test")
print(metrics.results_dict)

#metrics = model.val(data=config_path, split="train")
#print(metrics.results_dict)

In [ ]:
import os
from ultralytics import YOLO

input_folder = "/kaggle/working/test/images"
output_folder = "/kaggle/working/predictions"

os.makedirs(output_folder, exist_ok=True)

supported_formats = (".jpg", ".jpeg", ".png", ".bmp")  # Formatos suportados
image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(supported_formats)]

for image_file in image_files:
    input_path = os.path.join(input_folder, image_file)

    model.predict(
        source=input_path, 
        save_txt=True,
        conf=0.5,
        project=output_folder,
        iou=0.6,
        name=os.path.splitext(image_file)[0],
        show_labels=False,
        device=0
    )

In [ ]:
import os
import cv2

predictions_folder = "/kaggle/working/predictions"
output_folder = "/kaggle/working/predictions_with_boxes"
input_images_folder = "/kaggle/working/test/images"

os.makedirs(output_folder, exist_ok=True)

for subdir in os.listdir(predictions_folder):
    subdir_path = os.path.join(predictions_folder, subdir)
    if not os.path.isdir(subdir_path):
        continue

    label_path = os.path.join(subdir_path, "labels", f"{subdir}.txt")
    image_path = os.path.join(input_images_folder, f"{subdir}.png")

    if not os.path.exists(label_path):
        print(f"Arquivo de labels ausente para: {subdir}")
        continue
    if not os.path.exists(image_path):
        print(f"Imagem original ausente para: {subdir}")
        continue

    image = cv2.imread(image_path)
    height, width, _ = image.shape

    with open(label_path, "r") as f:
        for line in f:
            # Ler valores no formato [classe, x_centro, y_centro, largura, altura]
            cls, x_center, y_center, bbox_width, bbox_height = map(float, line.split())

            # Converter coordenadas normalizadas para valores absolutos
            x1 = int((x_center - bbox_width / 2) * width)
            y1 = int((y_center - bbox_height / 2) * height)
            x2 = int((x_center + bbox_width / 2) * width)
            y2 = int((y_center + bbox_height / 2) * height)

            # Desenhar a bounding box
            start_point = (x1, y1)
            end_point = (x2, y2)
            color = (0, 255, 0)  # Cor da bounding box (verde)
            thickness = 2
            image = cv2.rectangle(image, start_point, end_point, color, thickness)

    # Salvar a imagem anotada
    output_image_path = os.path.join(output_folder, f"{subdir}.jpg")
    cv2.imwrite(output_image_path, image)
    print(f"Imagem anotada salva em: {output_image_path}")

print("Processamento concluído!")

In [ ]:
import os

# Função para calcular o IoU
def calculate_iou(box1, box2):
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)
    inter_width = max(0, inter_x_max - inter_x_min)
    inter_height = max(0, inter_y_max - inter_y_min)
    inter_area = inter_width * inter_height
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area > 0 else 0

# Caminhos para ground truth e predições
ground_truth_folder = "/kaggle/working/test/labels"  # Pasta com o ground truth
predictions_folder = "/kaggle/working/predictions"  # Pasta com as predições
input_images_folder = "/kaggle/working/test/images"  # Onde estão as imagens originais
output_folder = "/kaggle/working/iou_output"  # Pasta para salvar as imagens com boxes

# Certifique-se de que a pasta de saída existe
os.makedirs(output_folder, exist_ok=True)

# IoU threshold para considerar uma detecção como correta
iou_threshold = 0.5

# Inicializa contadores
true_positives = 0
false_positives = 0
false_negatives = 0

# Loop pelos arquivos do ground truth
for gt_file in os.listdir(ground_truth_folder):
    gt_path = os.path.join(ground_truth_folder, gt_file)
    
    # A predição está dentro de uma subpasta da mesma imagem no diretório predictions
    pred_subfolder = os.path.join(predictions_folder, os.path.splitext(gt_file)[0])  # Imagem sem extensão
    pred_path = os.path.join(pred_subfolder, "labels", gt_file)

    # Verifica se a predição existe para o ground truth
    if not os.path.exists(pred_path):
        print(f"Predição ausente para: {gt_file}")
        continue

    # Verifica se o arquivo de imagem existe e se é um arquivo válido
    image_name = os.path.splitext(gt_file)[0]  # Remove a extensão do nome da imagem
    image_path = os.path.join(input_images_folder, image_name + ".png")  # Tentando com .jpg, ou você pode ajustar para .png, etc.
    
    if not os.path.exists(image_path):
        print(f"Imagem ausente ou tipo de arquivo incorreto: {image_path}")
        continue

    # Lê a imagem original
    image = cv2.imread(image_path)
    if image is None:
        print(f"Não foi possível carregar a imagem: {image_path}")
        continue

    height, width, _ = image.shape

    # Lê os bounding boxes do ground truth
    with open(gt_path, "r") as f:
        gt_boxes = [
            list(map(float, line.strip().split()[1:]))  # Ignora a classe
            for line in f
        ]
    
    # Verifica se as caixas do ground truth estão sendo lidas corretamente
    if not gt_boxes:
        print(f"Sem boxes no arquivo de ground truth: {gt_file}")
        continue

    # Lê os bounding boxes das predições
    with open(pred_path, "r") as f:
        pred_boxes = [
            list(map(float, line.strip().split()[1:]))  # Ignora a classe
            for line in f
        ]
    
    # Verifica se as caixas de predição estão sendo lidas corretamente
    if not pred_boxes:
        print(f"Sem boxes no arquivo de predição: {pred_path}")
        continue

    # Para cada box no ground truth, verifica se foi detectado
    matched = set()
    for gt_box in gt_boxes:
        gt_x_center, gt_y_center, gt_width, gt_height = gt_box
        gt_x1 = (gt_x_center - gt_width / 2) * width
        gt_y1 = (gt_y_center - gt_height / 2) * height
        gt_x2 = (gt_x_center + gt_width / 2) * width
        gt_y2 = (gt_y_center + gt_height / 2) * height

        gt_box_abs = [gt_x1, gt_y1, gt_x2, gt_y2]

        # Verifica se as coordenadas estão dentro dos limites da imagem
        if gt_x1 < 0 or gt_y1 < 0 or gt_x2 > width or gt_y2 > height:
            print(f"Coordenadas do GT fora dos limites da imagem: {gt_file}")
            continue

        # Desenha a caixa do ground truth somente se não for um verdadeiro positivo
        start_point = (int(gt_x1), int(gt_y1))
        end_point = (int(gt_x2), int(gt_y2))
        color = (255, 0, 0)  # Azul para o ground truth
        thickness = 1

        # Verifica as predições para esse ground truth
        detected = False
        for i, pred_box in enumerate(pred_boxes):
            if i in matched:
                continue

            pred_x_center, pred_y_center, pred_width, pred_height = pred_box
            pred_x1 = (pred_x_center - pred_width / 2) * width
            pred_y1 = (pred_y_center - pred_height / 2) * height
            pred_x2 = (pred_x_center + pred_width / 2) * width
            pred_y2 = (pred_y_center + pred_height / 2) * height

            pred_box_abs = [pred_x1, pred_y1, pred_x2, pred_y2]

            # Verifica se as coordenadas de predição estão dentro da imagem
            if pred_x1 < 0 or pred_y1 < 0 or pred_x2 > width or pred_y2 > height:
                print(f"Coordenadas da predição fora dos limites da imagem: {gt_file}")
                # Corrige as coordenadas para ficarem dentro dos limites
                pred_x1 = max(0, pred_x1)
                pred_y1 = max(0, pred_y1)
                pred_x2 = min(width, pred_x2)
                pred_y2 = min(height, pred_y2)

            # Calcula o IoU
            iou = calculate_iou(gt_box_abs, pred_box_abs)
            if iou >= iou_threshold:
                matched.add(i)
                true_positives += 1

                # Desenha a caixa da predição correta (True Positive)
                start_point = (int(pred_x1), int(pred_y1))
                end_point = (int(pred_x2), int(pred_y2))
                color = (0, 255, 0)  # Verde para acertos
                image = cv2.rectangle(image, start_point, end_point, color, thickness)
                detected = True
                break

        # Se não foi detectado, desenha a caixa do ground truth
        if not detected:
            false_negatives += 1
            image = cv2.rectangle(image, start_point, end_point, color, thickness)

    # Verifica predições não correspondentes (False Positives)
    for i, pred_box in enumerate(pred_boxes):
        if i not in matched:
            false_positives += 1
            pred_x_center, pred_y_center, pred_width, pred_height = pred_box
            pred_x1 = (pred_x_center - pred_width / 2) * width
            pred_y1 = (pred_y_center - pred_height / 2) * height
            pred_x2 = (pred_x_center + pred_width / 2) * width
            pred_y2 = (pred_y_center + pred_height / 2) * height

            # Garantir que as coordenadas de predição estão dentro dos limites da imagem
            pred_x1 = max(0, pred_x1)
            pred_y1 = max(0, pred_y1)
            pred_x2 = min(width, pred_x2)
            pred_y2 = min(height, pred_y2)

            # Desenha a caixa do False Positive
            start_point = (int(pred_x1), int(pred_y1))
            end_point = (int(pred_x2), int(pred_y2))
            color = (0, 0, 255)  # Vermelho para predições erradas
            image = cv2.rectangle(image, start_point, end_point, color, thickness)

    # Salva a imagem com as caixas desenhadas
    output_image_path = os.path.join(output_folder, image_name + "_output.jpg")
    cv2.imwrite(output_image_path, image)

# Imprime estatísticas
print(f"True Positives: {true_positives}")
print(f"False Positives: {false_positives}")
print(f"False Negatives: {false_negatives}")

In [ ]:
import os
import shutil

folders_to_zip = ['/kaggle/working/test', '/kaggle/working/iou_output', '/kaggle/working/predictions']

temp_dir = '/kaggle/working/zip_temp'

# Caminho final do arquivo zip
zip_path = '/kaggle/working/output.zip'

if os.path.exists(temp_dir):
    shutil.rmtree(temp_dir)
os.makedirs(temp_dir)

for folder in folders_to_zip:
    folder_name = os.path.basename(folder)
    shutil.copytree(folder, os.path.join(temp_dir, folder_name))

# Compactar a pasta temporária
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', temp_dir)

shutil.rmtree(temp_dir)

In [ ]:
import os
import numpy as np

# Caminhos
pred_dir = "/kaggle/working/predictions"
gt_dir = "/kaggle/working/test/labels" 

# Função para calcular IoU
def compute_iou(box1, box2):
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2

    xi1 = max(x1_min, x2_min)
    yi1 = max(y1_min, y2_min)
    xi2 = min(x1_max, x2_max)
    yi2 = min(y1_max, y2_max)

    inter_area = max((xi2 - xi1), 0) * max((yi2 - yi1), 0)
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = box1_area + box2_area - inter_area

    if union_area == 0:
        return 0.0
    return inter_area / union_area

# Função para converter YOLO para box absoluto
def yolo_to_box(line, img_w=640, img_h=640):
    _, x_c, y_c, w, h = map(float, line.strip().split())
    x_c *= img_w
    y_c *= img_h
    w *= img_w
    h *= img_h
    x_min = x_c - w / 2
    y_min = y_c - h / 2
    x_max = x_c + w / 2
    y_max = y_c + h / 2
    return [x_min, y_min, x_max, y_max]

# Verificação por imagem
perfect_matches = 0
total_images = 0

for img_folder in os.listdir(pred_dir):
    pred_file = os.path.join(pred_dir, img_folder, "labels", f"{img_folder}.txt")
    gt_file = os.path.join(gt_dir, f"{img_folder}.txt")
    
    if not os.path.exists(pred_file) or not os.path.exists(gt_file):
        continue

    with open(pred_file, "r") as f:
        preds = [yolo_to_box(line) for line in f.readlines()]
    with open(gt_file, "r") as f:
        gts = [yolo_to_box(line) for line in f.readlines()]

    matched = [False] * len(gts)

    for gt_idx, gt_box in enumerate(gts):
        for pred_box in preds:
            iou = compute_iou(gt_box, pred_box)
            if iou >= 0.5:
                matched[gt_idx] = True
                break

    if all(matched):
        perfect_matches += 1
    total_images += 1

print(f"Imagens com 100% de acerto: {perfect_matches}/{total_images}")